# Db2 on Cloud REST API


APIDoc: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4

こちらは[IBM Database Dojo Db2 REST APIを使ってみよう! 〜Db2 on Cloud編〜](https://ibm-developer.connpass.com/event/288544/)で使用するnotebookです

# 0. 前準備

必要なライブラリの導入やimportをまとめて行います。
セルを実行してください。


**実行は　セルを選択して**<br>
**Windows： Ctrl+Enter**<br>
**Mac: ⌘ (command) + Enter 　または Ctrl + Enter**<br>

In [ ]:
# 必要なイブラリのインストール
!pip install japanize-matplotlib #日本語グラフ用

In [ ]:
# 必要なイブラリのimport (グラフ用のみ4-3でimport)
import requests
import sys
import pandas as pd
import json

# 1. 必要な情報を入手しましょう！

- Db2 REST Service URL 
  - Db2 on Cloud Web UIの「管理」→ 「接続」から取得

- ターゲットのDb2 on CloudのCRN
  - IBM Cloudのリソースリスト　または　 
  - Db2 on Cloud Web UIの「製品情報」の「名前」から取得

- ターゲットのDb2 on Cloudの
    - useridとパスワード
     - hostname, port, db 名

詳細はQiita[Db2 on CloudのREST APIを使用したデータロード (python & curl)「1. 前準備」](https://qiita.com/nishikyon/items/cf45ffa83334444674dc#1-%E5%89%8D%E6%BA%96%E5%82%99)を参照してください。

取得した情報を以下の変数にセットします。<>で囲まれた部分を取得した値に置き換えてください。
入力完了後、セルの実行をしてください。




In [ ]:
#必要な変数のセット
REST_API_URL = "<Db2 REST Service URL>"
CRN = "<ターゲットのDb2 on CloudのCRN>"
DB_USERID = "<dbのuserid>"
DB_PW = "<dbのpassword>"

確認

以下のセルを実行してセットした値が表示されればOKです。

In [ ]:
print(f'REST_API_URL: {REST_API_URL}')
print(f'CRN: {CRN}')
print(f'DB_USERID: {DB_USERID}')
print(f'DB_PW: {DB_PW}')

# 2. アクセストークンの取得
APIDoc: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4#authenticate に従って、アクセストークンを取得します。


In [ ]:
#REST API の　　URL 作成
service_name = '/dbapi/v4/auth/tokens'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['x-deployment-id']= CRN


# parameter dbアクセス用のuid, pwを指定
params = {}
params['userid'] = DB_USERID
params['password']= DB_PW

# RESTserviceの呼び出し　POST
try:
    r = requests.post(url, headers=headers,  json=params)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)
    sys.exit()

#  Invalid credentials　エラーの確認
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)
    sys.exit()

# 401または200以外の場合は何かエラーがあるので確認(401は上で確認済み)
if (r.status_code != 200): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.json())
    sys.exit()

# access tokenの取得
try:
    access_token = r.json()['token'] 
    #print(r.json())
except:
    print("RESTful call did not return an access token.")
    print(r.json())
    sys.exit()

# access_token　の表示
print (access_token)

#  3. 試しにDB情報を取得してみる

- Get system information
  - https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4?code=python#getaboutinfo


In [ ]:
#REST API の　　URL 作成
service_name = f'/dbapi/v4/about'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['authorization'] =  'Bearer ' + access_token #アクセストークンをHeaderにセット
headers ['x-deployment-id'] = CRN

# RESTserviceの呼び出し　GET
try:
    r = requests.get(url, headers=headers)
   # print( r.status_code)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)

# Invalid credentials　エラーの確認
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)
    
# 401または200以外の場合は何かエラーがあるので確認(401は上で確認済み)
elif (r.status_code != 200): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.text)

else:
     print(json.dumps(r.json(), indent=4))

# 

# 4. SQLの実行 
API: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4#runsql

[東京都オープンデータカタログサイト](https://portal.data.metro.tokyo.lg.jp/)に「[東京都卸売市場日報（2022年度）](https://catalog.data.metro.tokyo.lg.jp/dataset/t000013d0000000005)」というデータがあります。

今回はこの中から[東京都卸売市場日報（販売予定数量・水産)](https://catalog.data.metro.tokyo.lg.jp/dataset/t000013d0000000005/resource/90f272ac-ef4c-4a9b-ad82-b4e183becfdf)のcsvファイルをDb2上のテーブルにロードしています。

東京都卸売市場日報（販売予定数量・水産)から、月毎の「まぐろ（生鮮）」の「卸売予定数量(t)」を出してみます。


手順は以下となります(2ステップとなります)
1. SQLの実行をリクエストし、JOBIDを取得
1. JOBIDを元に結果を取得



## 4-1. SQLの実行

API Doc: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4#runsql

バックグラウンド・ジョブとして 1 つ以上の SQL 文を実行します。このエンドポイントは、結果を取得するために使用できるジョブ ID を返します。

東京都卸売市場日報（販売予定数量・水産)から、月毎の「まぐろ（生鮮）」の「卸売予定数量(t)」を出すSQL:
```
SELECT TO_CHAR("日付",'yyyy-mm') AS "年月", sum("卸売予定数量(t)") AS "卸売予定数量(t)" 
FROM TOKYO_FISH 
WHERE "品名" = 'まぐろ（生鮮）'
GROUP BY TO_CHAR("日付",'yyyy-mm') ;
```

In [ ]:
# Executes SQL statements
#  SQLの実行
#  API Doc: API Doc: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4#runsql


#REST API の　　URL 作成
service_name = '/dbapi/v4/sql_jobs'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['authorization'] =  'Bearer ' + access_token #アクセストークンをHeaderにセット
headers ['x-deployment-id'] = CRN

# SQL文とオプションのセット
data = {
  #SQL文
  "commands": 'SELECT TO_CHAR("日付",\'yyyy-mm\') AS "年月", sum("卸売予定数量(t)") AS "卸売予定数量(t)" \
    FROM TOKYO_FISH WHERE "品名" = \'まぐろ（生鮮）\' \
        GROUP BY TO_CHAR("日付", \'yyyy-mm\') ;',
  #各結果セットでフェッチされる行の最大数
  "limit"  : 1000000,
  #SQL スクリプトに複数のステートメントが含まれている場合に、SQL ステートメントの終了を示すために使用される文字
   "separator": ";",
   #yes'の場合、ジョブはエラーを返す最初のステートメントで実行を停止する。no' の場合、1つ以上のステートメントがエラーを返してもジョブの実行は継続される。
   "stop_on_error":"no"
}

# RESTserviceの呼び出し　POST
try:
    r = requests.post(url, headers=headers, json=data)
    print( r.status_code)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)


# Invalid credentials　エラーの確認
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)


# 401または201以外の場合は何かエラーがあるので確認(401は上で確認済み)
elif (r.status_code != 201): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.text)
    #print(json.dumps(r.json(), indent=4))


# レスポンスを出力
else:
    print(json.dumps(r.json(), indent=4))
    JOB_ID = r.json()["id"] #このldの値は次のRESTAPIで使用する
    print(f"JOB_ID: {JOB_ID}" )   


## 4-2. SQL実行結果の取得

APIDoc: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4#fetchsqlresults

- SQL ジョブの実行の現在のステータスと、すでに実行された SQL 文の結果を返します。
- クライアントは、返されるステータスが 'completed' (すべての SQL 文の実行が完了したことを示す) か 'failed' (ジョブの実行に失敗したため終了したと見なされる) のいずれかになるまで、このエンドポイントをポーリングすることになっています。
- 返される結果のリストは累積されません。つまり、以前の呼び出しで取得された結果は再度返されず、新しい結果（つまり、まだ取得されていない結果）のみが含まれます。
  - 例えば、10個のSQL文を持つジョブを想定すると、最初の呼び出しはステータス "running "と6個の結果を返し、2回目の呼び出しはステータス "running "と空の結果リストを返し、3回目の呼び出しはステータス "completed "と4個の結果を返します。それ以降の呼び出しは、ステータス "completed "と空の結果リストを返します。  

In [ ]:
# Fetches partial results of a SQL job execution
#   SQL実行結果の取得
#  API Doc: https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4#fetchsqlresults
# Stasuがcompletedになるまで何度も実行


#REST API の　　URL 作成
service_name = f'/dbapi/v4/sql_jobs/{JOB_ID}'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['authorization'] =  'Bearer ' + access_token #アクセストークンをHeaderにセット
headers ['x-deployment-id'] = CRN

# RESTserviceの呼び出し　GET
try:
    r = requests.get(url, headers=headers)
   # print( r.status_code)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)

# Invalid credentials　エラーの確認
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)
    
# 401または200以外の場合は何かエラーがあるので確認(401は上で確認済み)
elif (r.status_code != 200): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.text)

else:
    print(f'Status: {r.json()["status"]}')
    # SQL文は1つのみの想定
    if r.json()["status"] != "failed":
        if len(r.json()["results"]) > 0:
            if 'error' in r.json()["results"][0] : #エラーの場合はresponseメッセージを表示
                print(json.dumps(r.json(), indent=4))
            else: 
                #結果をpandasのDataFrameに入れる, SQL文は1つのみの想定
                #print(json.dumps(r.json(), indent=4))
                df = pd.DataFrame(data=r.json()["results"][0]["rows"] , columns =r.json()["results"][0]["columns"])
                # 結果の表示
                display(result_df)
        else:
            # 結果セットがない場合はresponseメッセージを表示
            print(json.dumps(r.json(), indent=4))
    else:
        # failの場合はresponseメッセージを表示
        print(json.dumps(r.json(), indent=4))
        


## 4-3.せっかくなのでグラフにしてみます

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

df["卸売予定数量(t)"] = pd.to_numeric(df["卸売予定数量(t)"])

ax=df.plot.bar(x="年月", y="卸売予定数量(t)", figsize=(20,8))

ax.set_title("まぐろ(生鮮)卸売予定数量(t)")
plt.show()